In [ ]:
import plonk
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plonk.__version__

## Snapshots

In [ ]:
filename = 'disc_00030.h5'
snap = plonk.load_snap(filename)

In [ ]:
snap['position']

In [ ]:
snap.loaded_arrays()

In [ ]:
snap.available_arrays()

In [ ]:
# You can also define your own alias to access arrays. For example, 
# if you prefer to use the name ‘coordinate’ rather than ‘position’, use the add_alias() method to add an alias.

snap.add_alias(name='position', alias='coordinate')
snap['coordinate']

In [ ]:
# The Snap object has a properties attribute which is a dictionary of metadata, i.e. non-array data, on the 
# snapshot.


snap.properties['time']

In [ ]:
# Units are available. We make use of the Python units library Pint. 
# The code units of the data are available as code_units.

snap.code_units['length']

In [ ]:
# You can set default units as follows.

snap.set_units(position='au', density='g/cm^3', velocity='km/s')

In [ ]:
snap['position']

In [ ]:
# Sink particles are handled separately from the fluid, e.g. gas or dust, particles. 
# They are available as an attribute sinks.

list(snap.properties)

In [ ]:
snap.sinks

In [ ]:
snap.sinks['spin']

# Simulation

In [ ]:
prefix = 'disc'
sim = plonk.load_simulation(prefix=prefix, directory='/home/faride/plonk_example_data/')

In [ ]:
sim.snaps[:5]

In [ ]:
#SPH simulation data also include auxiliary files containing globally-averaged quantities output more frequently than 
#snapshot files. For example, Phantom writes text files with the suffix .ev. 
#These files are output every time step rather than at the frequency of the snapshot files.
#The Plonk Evolution class encapsulates this data. Use load_ev() to instantiate.

ev = plonk.load_time_series('/home/faride/plonk_example_data/disc01.ev')

In [ ]:
# if the simulation was run with multiple jobs on a computation cluster:
# In that case, pass in a tuple or list of files in chronological order to load_ev(), 
# and Plonk will concatenate the data removing any duplicated time steps.
# The underlying data is stored as a pandas DataFrame

ev.columns

In [ ]:
# You can plot columns with the pandas plotting interface.

ev.plot('time', ['density_max', 'center_of_mass_x', 'center_of_mass_z'])

In [ ]:
# SPH simulation datasets often include auxiliary files containing globally-averaged time series data output more
# frequently than snapshot files. For example, Phantom writes text files with the file extension “.ev”. 
# These files are output every time step rather than at the frequency of the snapshot files.

ts = plonk.load_time_series('disc01.ev')

In [ ]:
ts

In [ ]:
# You can plot columns with the pandas plotting interface.

ts.plot('time', ['center_of_mass_x', 'center_of_mass_y', 'center_of_mass_z'])
# The accretion disc center of mass as a function of time.

In [ ]:
# Visualization of SPH data

In [ ]:
# SPH particle data is not gridded like the data produced by, for example, finite difference or finite volume 
# hydrodynamical codes. One visualization method is to plot the particles as a scatter plot, and possibly color 
# the particles with the magnitude of a quantity of interest. An alternative is to interpolate any quantity on 
# the particles to a pixel grid with weighted kernel density estimation. This is what Splash does. 
# For the technical details, see Price (2007), PASA, 24, 3, 159. We use the same numerical method as Splash, 
# with the Python function compiled with Numba so it has the same performance as the Fortran code.

snap.image(quantity='density')

In [ ]:
# Alternatively, you can pass keyword arguments to the matplotlib functions.

snap.set_units(position='au', density='g/cm^3', projection='cm')
snap.image(
    quantity='density',
    extent=(20, 120, -50, 50),
    cmap='gist_heat',
    vmin=0.1,
    vmax=0.2,
)    

# The column density zoomed around the planet.

In [ ]:
# SPH particle data is not gridded like the data produced by, for example, finite difference or finite volume 
# hydrodynamical codes. One visualization method is to plot the particles as a scatter plot

#ax = plt.axes
#viz = plonk.visualize.plots(snap=snap, x='time', y='density_max', ax=ax)
extent = [-200, 200, -200, 200] * plonk.units('au')
ax = snap.image(quantity='density', extent=extent)
# we can use the Matplotlib image.AxesImage object to set the limits of the colorbar.

In [ ]:
# dir(module) returns the names of the attributes of the module

dir(plonk.visualize)

In [ ]:
# Analysis of SPH data

In [ ]:
# Analysis of SPH data
# If we want to visualize the subset of particles separately, for example dust and gas

gas = snap.family('gas')
dust = snap.family('dust', squeeze=True)

In [ ]:
# You can access arrays on the SubSnap objects as for any Snap object.

gas['mass'].sum().to('solar_mass')


In [ ]:
dust['mass'].sum().to('earth_mass')

In [ ]:
# to plot the gas and dust side-by-side

subsnaps = [gas, dust]
extent = (-200, 200, -200, 200)

fig, axs = plt.subplots(ncols=2, figsize=(14, 5))

for subsnap, ax in zip(subsnaps, axs):
    subsnap.image(quantity='density', extent=extent, cmap='gist_heat', ax=ax)
    
# the column density of gas and dust    

# derived arrays

In [ ]:
# Sometimes you need new arrays on the particles that are not available in the snapshot files

list(snap._file_pointer['particles'])

In [ ]:
# To see all available arrays on the Snap object:
​
snap.available_arrays()

In [ ]:
# You can add quantities appropriate for discs with the add_quantities()

previous_arrays = snap.available_arrays()
snap.add_quantities('disc')

In [ ]:
set(snap.available_arrays()) - set(previous_arrays)

In [ ]:
# if you want to create a new, derived array on the particles as follows.

snap['rad'] = np.sqrt(snap['x'] ** 2 + snap['y'] ** 2)
snap['rad']

In [ ]:
# Alternatively, you can define a function for a derived array. This makes use of the decorator add_array().

In [ ]:
# Show an image of the surface density in xy-plane.

plonk.image(snap=snap, quantity='density')

In [ ]:
snap.image(quantity='density')

In [ ]:
# Set units for the plot.

units = {'position': 'au', 'density': 'g/cm^3', 'projection': 'cm'}
snap.image(quantity='density', units=units)

In [ ]:
# Show a slice image of the density in xy-plane at z=0.

snap.image(quantity='density', interp='slice')

In [ ]:
# plonk.plot
# Show the particles in xy-plane

plonk.plot(snap=snap)

In [ ]:
# Alternatively, access the function as a method on the Snap object.

snap.plot()

In [ ]:
# Plot density against x.

snap.plot(x='x', y='density')

In [ ]:
# Color particles by density.

snap.plot(x='x', y='y', c='density')

In [ ]:
# Set units for the plot

units = {'position': 'au', 'density': 'g/cm^3'}
snap.plot(x='x', y='y', c='density', units=units)

In [ ]:
# plonk.vector

plonk.vector(snap=snap, quantity='velocity')

In [ ]:
# Alternatively

snap.vector(quantity='velocity')

In [ ]:
# Set units for the plot

units = {'position': 'au', 'velocity': 'km/s', 'projection': 'km'}
snap.vector(quantity='velocity', units=units)

In [ ]:
# Show a slice plot of the velocity in xy-plane at z=0.

snap.vector(quantity='velocity', interp='slice')

In [ ]:
dir(plonk.visualize_sim)

In [ ]:
# Alternatively

sim.visualize(kind='image', quantity='density')

# Units

In [ ]:
# Plonk uses Pint to set arrays to physical units.

snap = plonk.load_snap(filename)
snap['x']

In [ ]:
# It is easy to convert quantities to different units as required.

snap['x'].to('au')

# Profiles

In [ ]:
# Generating a profile is a convenient method to reduce the dimensionality of the full data set. 
# For example, we may want to see how the surface density and aspect ratio of the disc vary with radius.

# To do this we use the Profile class in the analysis module.

snap = plonk.load_snap(filename)
snap.add_quantities('disc')
prof = plonk.load_profile(snap, cmin='10 au', cmax='200 au')
prof

In [ ]:
# To see what profiles are loaded and 
# what are available use the loaded_profiles() and available_profiles() methods.

prof.loaded_profiles()

In [ ]:
prof.available_profiles()

In [ ]:
# To load a profile, simply call it.

prof['density']

In [ ]:
# You can convert the data in the Profile object to a pandas DataFrame with the to_dataframe() method. 
# This takes all loaded profiles and puts them into the DataFrame with units indicated in brackets.

profiles = [
    'density',
    'radius',
]
df = prof.to_dataframe(columns=profiles)
df

In [ ]:
# We can also plot the profiles.

prof.set_units(position='au', radius='au', density='kg/m^3')

fig, axs = plt.subplots(ncols=2, figsize=(12, 5))

prof.plot('radius', 'density', ax=axs[0])

prof.plot('radius', 'scale_height', ax=axs[1])


In [ ]:
# now form au to pc

snap = plonk.load_snap(filename)
snap['x'].to('pc')


# plonk.animate

In [ ]:
# plonk.animate

# for example: make an image animation of projected density 

FilePath = !ls ./disc_000*.h5
for Filelist in FilePath:
    print(Filelist)
    snaps = plonk.load_snap(Filelist)
    print(snaps)
units = {
        'position': 'au',
        'density': 'g/cm^3',
        'projection': 'cm'
        }
plonk.animate(
        filename='animation.mp4',
        snaps=snaps,
        quantity='density',
        units=units,
        save_kwargs={'fps': 10, 'dpi': 300}
    )    
             
    

In [ ]:
units={'position': 'au', 'surface_density': 'g/cm^2'}
plonk.animate(
    filename='animation.mp4',
    profiles=profiles,
    x='radius',
    y='surface_density',
    units=units,
    adaptive_limits=False,
    save_kwargs={'fps': 10, 'dpi': 300},
)